# Island Surface Defect Formation on TiN(001) Surface

## 0. Introduction

This tutorial demonstrates the process of creating a rectangular island defect on a slab. 


> **D. G. Sangiovanni, A. B. Mei, D. Edström, L. Hultman, V. Chirita, I. Petrov, and J. E. Greene**, "Effects of surface vibrations on interlayer mass transport: Ab initio molecular dynamics investigation of Ti adatom descent pathways and rates from TiN/TiN(001) islands", Physical Review B, 2018. [DOI: 10.1103/PhysRevB.97.035406](https://journals.aps.org/prb/abstract/10.1103/PhysRevB.97.035406).
> 

![Fig 2.](https://github.com/Exabyte-io/documentation/raw/12617167278ae3523adc028583b21ea4e8ebd197/images/tutorials/materials/defects/defect-creation-surface-island-titanium-nitride/0.png)

In this notebook we will reproduce material from FIG. 2 a)


## 1. Prepare the Environment
### 1.1. Set up defects parameters 



In [ ]:
NUMBER_OF_ADDED_LAYERS = 0.5

SHAPE_PARAMETERS = {
    'shape': 'box',
    'min_coordinate': [0.25, 0.2, 0],
    'max_coordinate': [0.65, 0.6, 1]
}

SLAB_PARAMETERS = {
    "miller_indices": (0, 0, 1),
    "thickness": 3,
    "vacuum": 10.0,
    "USE_ORTHOGONAL_C": True,
    "xy_supercell_matrix": [[10, 0], [0, 10]]
}

### 1.2. Install Packages
The step executes only in Pyodide environment. For other environments, the packages should be installed via `pip install` (see [README](../../README.ipynb)).

In [ ]:
import sys

if sys.platform == "emscripten":
    import micropip

    await micropip.install(
        "mat3ra-api-examples", deps=False)
    await micropip.install('mat3ra-utils')
    from mat3ra.utils.jupyterlite.packages import install_packages

    await install_packages("specific_examples")

### 1.3. Get input material
In this notebook we will use materials from Standata.

In [ ]:
from mat3ra.made.material import Material
from mat3ra.standata.materials import Materials

material = Material.create(Materials.get_by_name_first_match("TiN"))

### 1.4. Preview Slab or create a new one

In [ ]:
from mat3ra.made.tools.helpers import create_slab
from utils.visualize import visualize_materials

slab = create_slab(
    crystal=material,
    termination_top=None,  # Use default termination
    miller_indices=SLAB_PARAMETERS["miller_indices"],
    number_of_layers=SLAB_PARAMETERS["thickness"],
    vacuum=SLAB_PARAMETERS["vacuum"],
    use_orthogonal_c=SLAB_PARAMETERS["USE_ORTHOGONAL_C"],
    xy_supercell_matrix=SLAB_PARAMETERS["xy_supercell_matrix"]
)

visualize_materials(slab)

## 2. Create the Target Material

In [ ]:
from mat3ra.made.tools.entities.coordinate import BoxCoordinateCondition
from mat3ra.made.tools.helpers import create_island

coordinate_condition = BoxCoordinateCondition(
    min_coordinate=SHAPE_PARAMETERS['min_coordinate'],
    max_coordinate=SHAPE_PARAMETERS['max_coordinate']
)

slab_with_island = create_island(
    slab=slab,
    condition=coordinate_condition,
    number_of_added_layers=NUMBER_OF_ADDED_LAYERS,
    use_cartesian_coordinates=False,
)

## 3. Visualize the Result(s)

In [ ]:
visualize_materials([{"material": slab, "title": "Original material", "rotation": "-45x"},
                     {"material": slab_with_island,
                      "title": f"Material with Island Defect ({SHAPE_PARAMETERS['shape']})","rotation": "-45x"}],
                    # viewer="wave"
                    )

## 4. Pass data to the outside runtime

In [ ]:
from utils.jupyterlite import download_content_to_file, set_materials

set_materials(slab_with_island)
download_content_to_file(slab_with_island.to_json(), "TiN_slab_with_island.json")